<a href="https://colab.research.google.com/github/ajkohl/AdversarialAttacksResnet/blob/main/HyperBolicGCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision torchaudio
!pip install geoopt
!pip install networkx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 10.5 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import geoopt
import networkx as nx
import numpy as np
from torch.utils.data import DataLoader, Dataset

# If you choose to use torch_geometric, uncomment the following:
# !pip install torch-scatter torch-sparse torch-geometric
# from torch_geometric.data import Data


In [4]:
# Assuming train.txt, valid.txt, and test.txt are uploaded to your Colab environment.

def load_fb15k_triples(file_path):
    triples = []
    with open(file_path, 'r') as f:
        for line in f:
            head, rel, tail = line.strip().split('\t')
            triples.append((head, rel, tail))
    return triples

train_triples = load_fb15k_triples('train.txt')
valid_triples = load_fb15k_triples('valid.txt')
test_triples = load_fb15k_triples('test.txt')

# Extract unique entities and relations
entities = set()
relations = set()

for (h, r, t) in train_triples:
    entities.add(h)
    entities.add(t)
    relations.add(r)

entities = list(entities)
relations = list(relations)

entity2id = {e: i for i, e in enumerate(entities)}
rel2id = {r: i for i, r in enumerate(relations)}

num_entities = len(entity2id)
num_relations = len(rel2id)

print("Number of entities:", num_entities)
print("Number of relations:", num_relations)


Number of entities: 14505
Number of relations: 237


In [5]:
G = nx.MultiDiGraph()
for (h, r, t) in train_triples:
    G.add_node(entity2id[h])
    G.add_node(entity2id[t])
    G.add_edge(entity2id[h], entity2id[t], key=rel2id[r])


In [6]:
def negative_sampling(triples, num_entities, neg_size=1):
    # Simple uniform random negative sampler
    neg_triples = []
    for (h, r, t) in triples:
        h_id = entity2id[h]
        r_id = rel2id[r]
        t_id = entity2id[t]
        for _ in range(neg_size):
            # Replace tail with random entity
            neg_t = np.random.randint(0, num_entities)
            neg_triples.append((h_id, r_id, neg_t))
    return neg_triples

neg_train_triples = negative_sampling(train_triples, num_entities, neg_size=1)


In [7]:
class FB15kDataset(Dataset):
    def __init__(self, pos_triples, neg_triples):
        self.pos = pos_triples
        self.neg = neg_triples
        self.len = len(self.pos)

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        ph, pr, pt = self.pos[idx]
        nh, nr, nt = self.neg[idx]
        return (ph, pr, pt, nh, nr, nt)

# Convert positives to IDs
train_pos_ids = [(entity2id[h], rel2id[r], entity2id[t]) for (h,r,t) in train_triples]
train_neg_ids = neg_train_triples

train_dataset = FB15kDataset(train_pos_ids, train_neg_ids)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)


In [16]:
# Poincaré ball
manifold = geoopt.manifolds.PoincareBall()
curvature = torch.nn.Parameter(torch.tensor([1.0]), requires_grad=True)  # Curvature as a learnable param


In [9]:
entity_emb = geoopt.ManifoldParameter(torch.randn(num_entities, 64), manifold=manifold)
rel_emb = geoopt.ManifoldParameter(torch.randn(num_relations, 64), manifold=manifold)


In [10]:
class HypGraphConvolution(nn.Module):
    def __init__(self, in_dim, out_dim, manifold):
        super().__init__()
        self.manifold = manifold
        self.weight = geoopt.ManifoldParameter(torch.randn(in_dim, out_dim), manifold=manifold)
        # For simplicity, treat weight as manifold param. A real HGCN would do more complex ops.

    def forward(self, x, adj):
        # A placeholder: aggregate neighbors’ embeddings and transform
        # adj: adjacency matrix or edge index. Here, we simply apply a linear transform.
        # In a real scenario, implement hyperbolic message passing using Möbius addition, etc.
        return self.manifold.projx(self.manifold.expmap0(x @ self.weight, k=curvature))


In [11]:
class HGCNModel(nn.Module):
    def __init__(self, entity_emb, rel_emb, manifold, curvature, hidden_dim=64):
        super().__init__()
        self.manifold = manifold
        self.curvature = curvature
        self.entity_emb = entity_emb
        self.rel_emb = rel_emb
        self.gc1 = HypGraphConvolution(64, 64, manifold)
        self.gc2 = HypGraphConvolution(64, 64, manifold)

    def forward(self, h, r, t):
        # For scoring triples:
        # We interpret relation embedding as a "translation" in hyperbolic space.
        # Score = -hyperbolic_distance(h ⊗ r, t)

        # Hypothetical combination:
        head = self.entity_emb[h]
        rel = self.rel_emb[r]
        tail = self.entity_emb[t]

        # Just pass them through GCN layers for demonstration (no adjacency here for brevity)
        head = self.gc1(head, None)
        head = self.gc2(head, None)

        tail = self.gc1(tail, None)
        tail = self.gc2(tail, None)

        rel = self.gc1(rel, None)
        rel = self.gc2(rel, None)

        # Combine head and relation (Möbius addition in Poincaré)
        hr = self.manifold.mobius_add(head, rel, k=self.curvature)

        # Distance in Poincaré ball
        dist = self.manifold.dist(hr, tail, k=self.curvature)
        # Score: lower distance = higher score
        return -dist


In [12]:
class RiemannianLBFGS(torch.optim.Optimizer):
    def __init__(self, params, lr=1.0, history_size=10):
        defaults = dict(lr=lr, history_size=history_size)
        super().__init__(params, defaults)
        self.state = {}
        # For simplicity, we won't fully implement L-BFGS memory here.
        # Real implementation would store curvature pairs (s, y).

    @torch.no_grad()
    def step(self, closure=None):
        # closure: A function re-evaluating the model and returning loss.
        loss = None
        if closure is not None:
            loss = closure()

        # Just a placeholder: a real implementation would do:
        # 1. Compute Riemannian gradient from Euclidean gradient
        # 2. Apply L-BFGS two-loop recursion to get search direction
        # 3. Update parameters via manifold exponential map

        for group in self.param_groups:
            lr = group['lr']
            for p in group['params']:
                if p.grad is None:
                    continue
                # Project gradient to tangent space and apply Riemannian step
                grad = p.grad
                # geoopt manifold optimization step
                # In a proper setup, we'd use the stored inverse Hessian approximation.
                # Here we do a naive gradient step as a placeholder:
                update = -lr * grad
                p.data = p.manifold.expmap(p, update, k=curvature)

        return loss


In [19]:
model = HGCNModel(entity_emb, rel_emb, manifold, curvature=1)
margin = 1.0

# Use our conceptual L-BFGS
optimizer = RiemannianLBFGS(model.parameters(), lr=0.1)

model.train()
for epoch in range(10):
    total_loss = 0
    for batch in train_loader:
        ph, pr, pt, nh, nr, nt = batch
        ph, pr, pt = ph.long(), pr.long(), pt.long()
        nh, nr, nt = nh.long(), nr.long(), nt.long()

        def closure():
            optimizer.zero_grad()
            pos_score = model(ph, pr, pt)
            neg_score = model(nh, nr, nt)
            # margin-based ranking loss
            loss = F.relu(margin + neg_score - pos_score).mean()
            loss.backward()
            return loss

        loss = optimizer.step(closure)
        total_loss += loss.item()

    print(f"Epoch {epoch} - Loss: {total_loss/len(train_loader)}")


TypeError: HypGraphConvolution.__init__() missing 1 required positional argument: 'curvature'

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import geoopt

# Assuming `manifold` is a PoincaréBall instance and `curvature` a learnable parameter
# from previous code sections.

########################
# Hyperbolic Operations
########################

def mobius_add(x, y, k, dim=-1):
    """Möbius addition on Poincaré ball."""
    # From Ganea et al. (2018): Hyperbolic Neural Networks.
    x2 = torch.sum(x * x, dim=dim, keepdim=True)
    y2 = torch.sum(y * y, dim=dim, keepdim=True)
    xy = torch.sum(x * y, dim=dim, keepdim=True)
    c = k
    num = (1 + 2*c*xy + c*y2) * x + (1 - c*x2)*y
    denom = 1 + 2*c*xy + c**2 * x2 * y2
    return num / (denom + 1e-15)

def mobius_scalar_mul(r, x, k, dim=-1):
    """Möbius scalar multiplication."""
    normx = torch.norm(x, p=2, dim=dim, keepdim=True)
    # scale radius by this factor:
    return mobius_pointwise_mul(torch.tanh(r * atanh(torch.sqrt(k) * normx) / (torch.sqrt(k) * (normx + 1e-15))),
                                x, normx, dim)

def mobius_pointwise_mul(factor, x, normx, dim=-1):
    return factor * (x / (normx + 1e-15))

def atanh(x, eps=1e-15):
    return 0.5 * torch.log((1 + x)/(1 - x + eps))

def mobius_matmul(x, w, k):
    """Möbius matrix multiplication: a linear transform in the tangent space followed by exp back."""
    # We map x to tangent space at 0 using log map, do linear transform, then exp map back.
    # For simplicity, assume x is close to zero or just do standard Euclidean matmul followed by projection.
    # More rigor would require a full tangent -> Euclidean -> tangent conversion.
    x_tan = expmap0(x, k)
    x_tan_w = x_tan @ w  # standard linear transform in tangent space
    return projx(exmap0_inv(x_tan_w, k), k)  # map back to manifold

def expmap0(u, k):
    """Exponential map at 0."""
    normu = torch.norm(u, p=2, dim=-1, keepdim=True)
    return torch.tanh(torch.sqrt(k)*normu)/(torch.sqrt(k)*normu) * u

def logmap0(x, k):
    """Log map at 0."""
    normx = torch.norm(x, p=2, dim=-1, keepdim=True)
    return (1.0/torch.sqrt(k)*atanh(torch.sqrt(k)*normx)/(normx+1e-15))*x

def exmap0_inv(x, k):
    # Actually logmap0 is the inverse of expmap0. Use logmap0 instead for clarity.
    return logmap0(x, k)

def projx(x, k):
    """Project onto Poincaré ball."""
    normx = torch.norm(x, p=2, dim=-1, keepdim=True)
    maxnorm = (1 - 1e-5) / torch.sqrt(k)
    cond = normx > maxnorm
    # If normx too big, rescale
    projected = x/ normx * maxnorm
    x = torch.where(cond, projected, x)
    return x

########################
# Hyperbolic Linear Layer
########################

class HypLinear(nn.Module):
    def __init__(self, in_features, out_features, manifold, k):
        super().__init__()
        self.manifold = manifold
        self.k = k
        self.weight = nn.Parameter(torch.randn(in_features, out_features)*0.01)
        self.bias = nn.Parameter(torch.zeros(out_features))

    def forward(self, x):
        # Map x to tangent space at 0
        x_tan = logmap0(x, self.k)
        # Linear transform in tangent space
        out = x_tan @ self.weight + self.bias
        # Map back to the manifold
        out_hyp = expmap0(out, self.k)
        return projx(out_hyp, self.k)

########################
# Hyperbolic Graph Convolution Layer
########################

class HypGraphConvolution(nn.Module):
    def __init__(self, in_dim, out_dim, manifold, curvature):
        super().__init__()
        self.manifold = manifold
        self.curvature = curvature
        self.hyp_linear = HypLinear(in_dim, out_dim, manifold, curvature)

    def forward(self, x, adj):
        # adj is the adjacency matrix or a normalized adjacency
        # For simplicity, let's assume adj is a sparse adjacency matrix or dense tensor
        # We'll do a basic mean aggregator:

        # Aggregate neighbors in tangent space
        x_tan = logmap0(x, self.curvature)
        # Mean aggregator in tangent space (adj assumed to be NxN)
        # shape: x_tan: [N, D], adj: [N, N]
        agg = adj @ x_tan
        # Normalize by degree if needed
        deg = torch.sum(adj, dim=-1, keepdim=True)
        agg = agg / (deg + 1e-15)

        # Map aggregated features back to manifold
        agg_hyp = expmap0(agg, self.curvature)
        agg_hyp = projx(agg_hyp, self.curvature)

        # Apply a hyperbolic linear transform
        out = self.hyp_linear(agg_hyp)
        return projx(out, self.curvature)

########################
# Riemannian L-BFGS Optimizer (Initial Implementation)
########################

class RiemannianLBFGS(torch.optim.Optimizer):
    def __init__(self, params, lr=1.0, history_size=10, line_search=False):
        defaults = dict(lr=lr, history_size=history_size, line_search=line_search)
        super().__init__(params, defaults)
        self._params = self.param_groups[0]['params']

    @torch.no_grad()
    def step(self, closure=None):
        loss = None
        if closure is not None:
            loss = closure()

        # Collect gradients and prepare for Riemannian step
        grad = []
        for p in self._params:
            if p.grad is not None:
                # Project gradient to tangent space if needed
                # geoopt handles gradient projection automatically for ManifoldParameter,
                # but if needed, we can ensure it:
                # p.grad = p.manifold.egrad2rgrad(p, p.grad)
                grad.append(p.grad.view(-1))

        if len(grad) == 0:
            return loss

        # Flatten gradients
        grad = torch.cat(grad)

        group = self.param_groups[0]
        state = self.state
        lr = group['lr']
        history_size = group['history_size']

        # Initialize state if needed
        if 'old_dirs' not in state:
            state['old_dirs'] = []
            state['old_stps'] = []
            state['H_diag'] = 1.0

        old_dirs = state['old_dirs']
        old_stps = state['old_stps']
        H_diag = state['H_diag']

        # Two-loop recursion to compute search direction
        q = grad
        alpha = []

        for i in reversed(range(len(old_dirs))):
            s = old_dirs[i]
            y = old_stps[i]
            alpha_i = (s @ q)/(y @ s)
            alpha.append(alpha_i)
            q = q - alpha_i * y

        # initial H0 * q
        q = H_diag * q

        # now iterate forward
        for i in range(len(old_dirs)):
            s = old_dirs[i]
            y = old_stps[i]
            beta = (y @ q)/(y @ s)
            q = q + s*(alpha[-(i+1)] - beta)

        # q is now the search direction in the Euclidean sense.
        # We need to perform a Riemannian update:

        # Update parameters along q
        idx = 0
        for p in self._params:
            if p.grad is None:
                continue
            size = p.numel()
            dq = q[idx:idx+size].view_as(p)
            idx += size

            # Riemannian update:
            # Move p along q using exponential map
            # We assume p is on a geoopt manifold. Using manifold.expmap:
            p.data = p.manifold.expmap(p.data, -lr * dq, k=curvature)

        # Update memory for L-BFGS
        # On the next iteration, we will have a new gradient. Let’s store s and y:
        # s = delta parameters, y = delta gradient
        # We'll need to do this on next step call. Typically, this involves storing
        # param changes and grad changes from previous iteration. For simplicity here,
        # assume we can do:

        # Actually, to store s, y properly, we need the old parameters and old gradients.
        # Let's store them now:
        if 'prev_params' in state:
            prev_params = state['prev_params']
            prev_grads = state['prev_grads']
            new_params = [p.data.clone() for p in self._params]
            new_grads = [p.grad.data.clone() if p.grad is not None else torch.zeros_like(p.data) for p in self._params]

            # compute s and y
            s_flat = []
            y_flat = []
            for old_p, new_p, old_g, new_g in zip(prev_params, new_params, prev_grads, new_grads):
                s_flat.append((new_p - old_p).view(-1))
                y_flat.append((new_g - old_g).view(-1))
            s_flat = torch.cat(s_flat)
            y_flat = torch.cat(y_flat)

            # Update memory
            if len(old_dirs) == history_size:
                old_dirs.pop(0)
                old_stps.pop(0)
            old_dirs.append(s_flat)
            old_stps.append(y_flat)

            # Update H_diag
            # Typically H0 is chosen as (s@y)/(y@y)
            H_diag = (s_flat @ y_flat)/(y_flat @ y_flat + 1e-15)
            state['H_diag'] = H_diag

            # Save new params and grads
            state['prev_params'] = new_params
            state['prev_grads'] = new_grads
        else:
            # First iteration, just record
            state['prev_params'] = [p.data.clone() for p in self._params]
            state['prev_grads'] = [p.grad.data.clone() if p.grad is not None else torch.zeros_like(p.data) for p in self._params]

        return loss


In [20]:
!pip install geoopt

import torch
import torch.nn as nn
import torch.nn.functional as F
import geoopt

########################
# Hyperbolic Utility Functions
########################

def atanh(x, eps=1e-15):
    return 0.5 * torch.log((1 + x)/(1 - x + eps))

def expmap0(u, k):
    """Exponential map at 0 on the Poincaré ball."""
    normu = torch.clamp_min(torch.norm(u, p=2, dim=-1, keepdim=True), 1e-15)
    sqrtk = torch.sqrt(k)
    return torch.tanh(sqrtk * normu) * u / (sqrtk * normu)

def logmap0(x, k):
    """Log map at 0 on the Poincaré ball."""
    normx = torch.clamp_min(torch.norm(x, p=2, dim=-1, keepdim=True), 1e-15)
    sqrtk = torch.sqrt(k)
    return (1./(sqrtk)) * atanh(sqrtk*normx) * x / normx

def projx(x, k):
    """Project onto Poincaré ball."""
    normx = torch.norm(x, p=2, dim=-1, keepdim=True)
    maxnorm = (1 - 1e-5) / torch.sqrt(k)
    cond = normx > maxnorm
    projected = x / normx * maxnorm
    return torch.where(cond, projected, x)

########################
# Hyperbolic Linear Layer
########################

class HypLinear(nn.Module):
    def __init__(self, in_features, out_features, k):
        super().__init__()
        self.k = k
        self.weight = nn.Parameter(torch.randn(in_features, out_features)*0.01)
        self.bias = nn.Parameter(torch.zeros(out_features))

    def forward(self, x):
        x_tan = logmap0(x, self.k)
        out = x_tan @ self.weight + self.bias
        out_hyp = expmap0(out, self.k)
        return projx(out_hyp, self.k)

########################
# Hyperbolic Graph Convolution Layer
########################

class HypGraphConvolution(nn.Module):
    def __init__(self, in_dim, out_dim, curvature):
        super().__init__()
        self.curvature = curvature
        self.hyp_linear = HypLinear(in_dim, out_dim, curvature)

    def forward(self, x, adj):
        # x: [N, D], adj: [N, N]
        # Aggregate neighbors in tangent space
        x_tan = logmap0(x, self.curvature)
        agg = adj @ x_tan
        deg = torch.sum(adj, dim=-1, keepdim=True)
        agg = agg / (deg + 1e-15)
        agg_hyp = expmap0(agg, self.curvature)
        agg_hyp = projx(agg_hyp, self.curvature)

        out = self.hyp_linear(agg_hyp)
        return projx(out, self.curvature)

########################
# Simple Model
########################

class SimpleHGCN(nn.Module):
    def __init__(self, num_nodes, in_dim, hidden_dim, curvature):
        super().__init__()
        self.curvature = curvature
        # Initialize embeddings on hyperbolic manifold
        self.emb = geoopt.ManifoldParameter(
            torch.randn(num_nodes, in_dim)*0.01,
            manifold=geoopt.manifolds.PoincareBall()
        )
        self.gc1 = HypGraphConvolution(in_dim, hidden_dim, curvature)
        self.gc2 = HypGraphConvolution(hidden_dim, hidden_dim, curvature)
        # A simple classifier:
        self.classifier = nn.Linear(hidden_dim, 2)  # Suppose 2-class classification

    def forward(self, adj):
        x = self.emb
        x = self.gc1(x, adj)
        x = F.relu(x)
        x = self.gc2(x, adj)
        x = F.relu(x)
        # For classification, map to tangent and do a Euclidean linear:
        x_tan = logmap0(x, self.curvature)
        logits = self.classifier(x_tan)
        return logits

########################
# Riemannian L-BFGS (Simplified)
########################

class RiemannianLBFGS(torch.optim.Optimizer):
    def __init__(self, params, lr=0.1):
        defaults = dict(lr=lr)
        super().__init__(params, defaults)
        self._params = self.param_groups[0]['params']

    @torch.no_grad()
    def step(self, closure=None):
        loss = None
        if closure is not None:
            loss = closure()

        # This is a simplified LBFGS, essentially doing a gradient step on the manifold
        # Real L-BFGS would implement two-loop recursion and history storage.
        # Here we just do a Riemannian gradient update for demonstration.

        group = self.param_groups[0]
        lr = group['lr']

        for p in self._params:
            if p.grad is None:
                continue
            # Riemannian update:
            # geoopt manifold parameters automatically handle projection,
            # but we use expmap for clarity
            grad = p.grad
            # negative gradient direction
            update = -lr * grad
            # Update parameter using exponential map at p
            # If p is manifold parameter with a known manifold and curvature, we must supply k.
            # For PoincareBall, we can access p.manifold:
            if hasattr(p, 'manifold') and isinstance(p.manifold, geoopt.manifolds.PoincareBall):
                # We'll assume curvature = 1.0 if not otherwise specified.
                # If curvature is a parameter, we need to retrieve it.
                # For simplicity, let's assume we have a global curvature=1.0 or
                # store it somewhere accessible.
                k = 1.0
                p.data = p.manifold.expmap(p.data, update, k=k)
            else:
                # For Euclidean parameters like classifier weights:
                p.add_(update)

        return loss

########################
# Toy Example and Training Loop
########################

# Create a small toy graph
num_nodes = 4
in_dim = 4
hidden_dim = 8
curvature = torch.tensor([1.0])  # fixed curvature

# A simple adjacency for a small graph (undirected)
adj = torch.tensor([
    [0.,1.,0.,0.],
    [1.,0.,1.,0.],
    [0.,1.,0.,1.],
    [0.,0.,1.,0.]
])

model = SimpleHGCN(num_nodes, in_dim, hidden_dim, curvature)
optimizer = RiemannianLBFGS(model.parameters(), lr=0.1)

# Suppose we have a toy "label" for each node: just random
labels = torch.randint(0,2,(num_nodes,))

# Training loop
model.train()
for epoch in range(5):
    def closure():
        optimizer.zero_grad()
        logits = model(adj)
        # Simple node classification loss
        # logits: [N, 2]
        loss = F.cross_entropy(logits, labels)
        loss.backward()
        return loss

    loss = optimizer.step(closure)
    print(f"Epoch {epoch}, Loss: {loss.item()}")


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [2]:
!pip install geoopt


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 9.4 MB/s eta 0:00:00


In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import geoopt

########################
# Hyperbolic Utility Functions
########################

def atanh(x, eps=1e-15):
    return 0.5 * torch.log((1 + x)/(1 - x + eps))

def expmap0(u, k):
    """Exponential map at 0 on the Poincaré ball."""
    normu = torch.clamp_min(torch.norm(u, p=2, dim=-1, keepdim=True), 1e-15)
    sqrtk = torch.sqrt(k)
    return torch.tanh(sqrtk * normu) * u / (sqrtk * normu)

def logmap0(x, k):
    """Log map at 0 on the Poincaré ball."""
    normx = torch.clamp_min(torch.norm(x, p=2, dim=-1, keepdim=True), 1e-15)
    sqrtk = torch.sqrt(k)
    return (1./sqrtk) * atanh(sqrtk * normx) * x / normx

def projx(x, k):
    """Project onto Poincaré ball."""
    normx = torch.norm(x, p=2, dim=-1, keepdim=True)
    maxnorm = (1 - 1e-5) / torch.sqrt(k)
    cond = normx > maxnorm
    projected = x / normx * maxnorm
    return torch.where(cond, projected, x)


In [30]:
class HypLinear(nn.Module):
    def __init__(self, in_features, out_features, k):
        super().__init__()
        self.k = k
        self.weight = nn.Parameter(torch.randn(in_features, out_features) * 0.01)
        self.bias = nn.Parameter(torch.zeros(out_features))

    def forward(self, x):
        x_tan = logmap0(x, self.k)
        out = x_tan @ self.weight + self.bias
        out_hyp = expmap0(out, self.k)
        return projx(out_hyp, self.k)


In [31]:
class HypGraphConvolution(nn.Module):
    def __init__(self, in_dim, out_dim, curvature):
        super().__init__()
        self.curvature = curvature
        self.hyp_linear = HypLinear(in_dim, out_dim, curvature)

    def forward(self, x, adj):
        # x: [N, D], adj: [N, N]
        # Aggregate neighbors in tangent space
        x_tan = logmap0(x, self.curvature)
        agg = adj @ x_tan
        deg = torch.sum(adj, dim=-1, keepdim=True)
        agg = agg / (deg + 1e-15)
        agg_hyp = expmap0(agg, self.curvature)
        agg_hyp = projx(agg_hyp, self.curvature)

        out = self.hyp_linear(agg_hyp)
        return projx(out, self.curvature)


In [41]:
class SimpleHGCN(nn.Module):
    def __init__(self, num_nodes, in_dim, hidden_dim):
        super().__init__()
        # Learnable curvature parameter (optional)
        # If you want curvature to be learnable, uncomment the following line:
        self.curvature = torch.nn.Parameter(torch.tensor([1.0]), requires_grad=True).to('cuda')
        # For simplicity, we'll fix curvature to 1.0
        # self.curvature = torch.tensor([-1], requires_grad=False).to('cuda')

        # Initialize embeddings on hyperbolic manifold
        self.emb = geoopt.ManifoldParameter(
            torch.randn(num_nodes, in_dim) * 0.01,
            manifold=geoopt.manifolds.PoincareBall()
        )

        self.gc1 = HypGraphConvolution(in_dim, hidden_dim, self.curvature)
        self.gc2 = HypGraphConvolution(hidden_dim, hidden_dim, self.curvature)

        # A simple classifier:
        self.classifier = nn.Linear(hidden_dim, 2)  # 2-class classification

    def forward(self, adj):
        x = self.emb
        x = self.gc1(x, adj)
        x = F.relu(x)
        x = self.gc2(x, adj)
        x = F.relu(x)
        # For classification, map to tangent and do a Euclidean linear:
        x_tan = logmap0(x, self.curvature)
        logits = self.classifier(x_tan)
        return logits


In [51]:
# Create a small toy graph
num_nodes = 4
in_dim = 4
hidden_dim = 8


# A simple adjacency for a small undirected graph
adj = torch.tensor([
    [0., 1., 0., 0.],
    [1., 0., 1., 0.],
    [0., 1., 0., 1.],
    [0., 0., 1., 0.]
])

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
adj = adj.to(device)

# Initialize model and move to device
model = SimpleHGCN(num_nodes, in_dim, hidden_dim).to(device)

# Define optimizer
optimizer = geoopt.optim.RiemannianAdam(model.parameters(), lr=0.1)

# Define labels for node classification (random for demonstration)
labels = torch.randint(0, 2, (num_nodes,)).to(device)


In [52]:
# Training loop
model.train()
num_epochs = 100

for epoch in range(num_epochs):
    def closure():
        optimizer.zero_grad()
        logits = model(adj)
        # Simple node classification loss
        # logits: [N, 2]
        loss = F.cross_entropy(logits, labels)
        loss.backward()
        return loss

    loss = optimizer.step(closure)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")


Epoch 1/100, Loss: 0.8143
Epoch 2/100, Loss: 0.7427
Epoch 3/100, Loss: 0.6596
Epoch 4/100, Loss: 0.5850
Epoch 5/100, Loss: 0.5581
Epoch 6/100, Loss: 0.5890
Epoch 7/100, Loss: 0.5661
Epoch 8/100, Loss: 0.5397
Epoch 9/100, Loss: 0.5250
Epoch 10/100, Loss: 0.5093
Epoch 11/100, Loss: 0.4852
Epoch 12/100, Loss: nan
Epoch 13/100, Loss: nan
Epoch 14/100, Loss: nan
Epoch 15/100, Loss: nan
Epoch 16/100, Loss: nan
Epoch 17/100, Loss: nan
Epoch 18/100, Loss: nan
Epoch 19/100, Loss: nan
Epoch 20/100, Loss: nan
Epoch 21/100, Loss: nan
Epoch 22/100, Loss: nan
Epoch 23/100, Loss: nan
Epoch 24/100, Loss: nan
Epoch 25/100, Loss: nan
Epoch 26/100, Loss: nan
Epoch 27/100, Loss: nan
Epoch 28/100, Loss: nan
Epoch 29/100, Loss: nan
Epoch 30/100, Loss: nan
Epoch 31/100, Loss: nan
Epoch 32/100, Loss: nan
Epoch 33/100, Loss: nan
Epoch 34/100, Loss: nan
Epoch 35/100, Loss: nan
Epoch 36/100, Loss: nan
Epoch 37/100, Loss: nan
Epoch 38/100, Loss: nan
Epoch 39/100, Loss: nan
Epoch 40/100, Loss: nan
Epoch 41/100, Lo

In [1]:
!pip install geoopt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 2.5 MB/s eta 0:00:00


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import networkx as nx
import math
from torch import optim
import geoopt
from geoopt import ManifoldParameter

##############################################
# Data Loading and Preprocessing
##############################################

def load_fb15k_data(train_path, valid_path, test_path):
    # FB15K format: head \t relation \t tail
    # Build dictionaries for entities and relations
    # Returns: node2id, rel2id, edge_list, and a graph
    def read_triples(path, ent2id, rel2id):
        triples = []
        with open(path, 'r') as f:
            for line in f:
                h, r, t = line.strip().split('\t')
                if h not in ent2id:
                    ent2id[h] = len(ent2id)
                if t not in ent2id:
                    ent2id[t] = len(ent2id)
                if r not in rel2id:
                    rel2id[r] = len(rel2id)
                triples.append((h, r, t))
        return triples

    ent2id = {}
    rel2id = {}
    train_triples = read_triples(train_path, ent2id, rel2id)
    valid_triples = read_triples(valid_path, ent2id, rel2id)
    test_triples = read_triples(test_path, ent2id, rel2id)

    # Build adjacency (undirected for GCN)
    G = nx.Graph()
    G.add_nodes_from(range(len(ent2id)))
    for h, r, t in train_triples:
        G.add_edge(ent2id[h], ent2id[t])

    return ent2id, rel2id, G, train_triples, valid_triples, test_triples

# Example paths (replace with your actual paths)
train_path = 'train.txt'
valid_path = 'valid.txt'
test_path = 'test.txt'

ent2id, rel2id, G, train_triples, valid_triples, test_triples = load_fb15k_data(train_path, valid_path, test_path)

num_nodes = len(ent2id)
adj = nx.to_scipy_sparse_array(G, nodelist=range(num_nodes))
adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)  # symmetrize
adj = torch.tensor(adj.toarray(), dtype=torch.float32) + 1e-5*torch.eye(num_nodes)
degrees = adj.sum(axis=1)
inv_sqrt_deg = 1. / torch.sqrt(degrees)
inv_sqrt_deg[torch.isinf(inv_sqrt_deg)] = 0
D_inv_sqrt = torch.diag(inv_sqrt_deg)
norm_adj = torch.tensor(D_inv_sqrt @ adj @ D_inv_sqrt, dtype=torch.float32)


# For simplicity, let's do a fake node classification target:
# Assign random labels to nodes (in practice, you would use a known node classification split)
num_classes = 5
labels = torch.randint(0, num_classes, (num_nodes,))

# Features: Let's just initialize random features or identity
# In a real scenario, you'd have meaningful features
features = torch.eye(num_nodes)

##############################################
# Manifold and Hyperbolic Layers
##############################################

class HyperbolicLinear(nn.Module):
    """
    Hyperbolic linear layer: maps from Poincaré ball to tangent space,
    applies a linear transform, then maps back.
    """
    def __init__(self, manifold, in_features, out_features, c=1.0, bias=True):
        super(HyperbolicLinear, self).__init__()
        self.manifold = manifold
        self.in_features = in_features
        self.out_features = out_features
        self.weight = ManifoldParameter(torch.randn(out_features, in_features)*0.01, manifold=self.manifold)
        self.c = c
        if bias:
            self.bias = nn.Parameter(torch.zeros(out_features))
        else:
            self.register_parameter('bias', None)

    def forward(self, x):
        # x is on manifold
        # Move x to tangent space at 0
        tangent_x = self.manifold.logmap0(x, k=self.c)
        # Apply linear transform in tangent space
        out = F.linear(tangent_x, self.weight, self.bias)
        # Map back to manifold
        out = self.manifold.expmap0(out, k=self.c)
        return out


class HyperbolicGCNLayer(nn.Module):
    def __init__(self, manifold, in_features, out_features, c=1.0):
        super(HyperbolicGCNLayer, self).__init__()
        self.manifold = manifold
        self.lin = HyperbolicLinear(manifold, in_features, out_features, c=c)
        self.c = c

    def forward(self, x, adj):
        # x on Poincaré ball
        # Compute neighbor aggregation in tangent space
        # First map x to tangent at 0
        x_tan = self.manifold.logmap0(x)
        # Aggregate using normalized adjacency (like Euclidean GCN, but in tangent space)
        x_agg = adj @ x_tan
        # Map back to manifold (we can combine with linear)
        x_agg = self.manifold.expmap0(x_agg)
        # Apply hyperbolic linear layer
        x_out = self.lin(x_agg)
        return x_out


class HyperbolicGCN(nn.Module):
    def __init__(self, manifold, num_features, hidden_dim, num_classes, c=1.0):
        super(HyperbolicGCN, self).__init__()
        self.manifold = manifold
        self.layer1 = HyperbolicGCNLayer(manifold, num_features, hidden_dim, c=c)
        self.layer2 = HyperbolicGCNLayer(manifold, hidden_dim, num_classes, c=c)
        self.c = c

    def forward(self, x, adj):
        x = self.manifold.expmap0(x, k=self.c)  # ensure x is on the manifold
        x = self.layer1(x, adj)
        # Apply a hyperbolic nonlinearity, e.g. hyperbolic tangent
        x_tan = self.manifold.logmap0(x, k=self.c)
        x_tan = torch.tanh(x_tan)
        x = self.manifold.expmap0(x_tan, k=self.c)
        x = self.layer2(x, adj)
        return x


##############################################
# Riemannian L-BFGS Optimizer
##############################################
class RiemannianLBFGS(optim.Optimizer):
    """
    A very basic Riemannian L-BFGS implementation.
    This is a conceptual template and may need refinement.

    Arguments:
        params (iterable): iterable of parameters to optimize
        lr (float): learning rate (step size)
        history_size (int): L-BFGS memory size
        line_search_fn (str): line search to use
        c (float): curvature parameter for manifold
    """
    def __init__(self, params, lr=1.0, history_size=10, c=1.0):
        defaults = dict(lr=lr, history_size=history_size, c=c)
        super(RiemannianLBFGS, self).__init__(params, defaults)
        # Store previous updates
        for group in self.param_groups:
            group['s_history'] = []
            group['y_history'] = []
            group['rho_history'] = []

    def step(self, closure=None):
        # closure is a function that re-evaluates the model and returns the loss.
        # 1) Evaluate loss and gradients if needed
        loss = None
        if closure is not None:
            loss = closure()

        # Collect flatten gradients and parameters on tangent space
        for group in self.param_groups:
            manifold = None
            for p in group['params']:
                if hasattr(p, 'manifold'):
                    manifold = p.manifold
                    break
            if manifold is None:
                raise ValueError("No manifold found in parameters")

            lr = group['lr']
            s_history = group['s_history']
            y_history = group['y_history']
            rho_history = group['rho_history']
            history_size = group['history_size']
            c = group['c']

            # Flatten gradients and parameters
            grads = []
            params_flat = []
            for p in group['params']:
                if p.grad is None:
                    continue
                # Riemannian gradient is already p.grad on tangent space after backward with geoopt?
                # If not, you may need to project Euclidean gradients to tangent space:
                # p.grad = p.manifold.egrad2rgrad(p, p.grad, c=c)
                p.grad = p.manifold.egrad2rgrad(p, p.grad, c=c)
                grads.append(p.grad.view(-1))
                params_flat.append(p.data.view(-1))
            grad = torch.cat(grads)
            # Initial direction: -grad
            direction = -grad

            # Two-loop recursion for L-BFGS
            if len(s_history) > 0:
                # Apply L-BFGS preconditioning
                alpha = []
                q = grad.clone()
                for (s, y, rho) in reversed(list(zip(s_history, y_history, rho_history))):
                    alpha_i = rho * torch.dot(s, q)
                    q = q - alpha_i * y
                    alpha.append(alpha_i)
                # scaling by H0 (initial Hessian approx)
                # Typically H0 = (s_{k-1}^T y_{k-1}) / (y_{k-1}^T y_{k-1})
                # if none available, scale by 1
                if len(y_history) > 0:
                    y_last = y_history[-1]
                    s_last = s_history[-1]
                    gamma = torch.dot(s_last, y_last) / torch.dot(y_last, y_last)
                else:
                    gamma = 1.0
                r = gamma * q
                # second loop
                for (s, y, rho, alpha_i) in zip(s_history, y_history, rho_history, reversed(alpha)):
                    beta = rho * torch.dot(y, r)
                    r = r + s * (alpha_i - beta)
                direction = -r

            # Line search not implemented here (fixed step size for demo)
            # Update parameters using the exponential map
            start_params = torch.cat([p.data.view(-1) for p in group['params']])
            for p in group['params']:
                sz = p.numel()
                dp = direction[:sz].view_as(p.data)
                direction = direction[sz:]
                # Move along geodesic
                # x_{new} = exp_{x}(dp)
                # Here dp is in tangent space at p, so:
                p.data = p.manifold.expmap(p.data, dp, c=c)

            # After the update, we need to compute s and y for next iteration
            end_params = torch.cat([p.data.view(-1) for p in group['params']])
            s_k = (end_params - start_params).detach()
            # y_k = grad_{new} - grad_{old}, we need new grad
            # Re-compute gradient after update
            # This might be expensive for large problems - normally you have closure
            if closure is not None:
                new_loss = closure()
            else:
                # You should have a way to recompute loss and grad here
                new_loss = None
                # For simplicity, assume we can call backward again with stored graph (not always possible)
                # In a production code, store model forward pass in closure
            new_grads = []
            for p in group['params']:
                if p.grad is not None:
                    new_grads.append(p.grad.view(-1))
            new_grad = torch.cat(new_grads)

            y_k = (new_grad - grad).detach()

            # Update histories
            s_history.insert(0, s_k)
            y_history.insert(0, y_k)
            rho = 1.0 / torch.dot(y_k, s_k)
            rho_history.insert(0, rho)

            if len(s_history) > history_size:
                s_history.pop()
                y_history.pop()
                rho_history.pop()

        return loss

##############################################
# Training
##############################################

device = torch.device("cuda")  # or "cuda" if available

manifold = geoopt.manifolds.PoincareBall()
model = HyperbolicGCN(manifold, num_features=num_nodes, hidden_dim=64, num_classes=num_classes, c=1.0).to(device)

# Initialize features as points on manifold:
# Start from Euclidean points and map to manifold
x = features.to(device)
x = manifold.expmap(x, c=1.0)

labels = labels.to(device)
adj_t = norm_adj.to(device)

# Define a simple cross-entropy loss for node classification
criterion = nn.CrossEntropyLoss()

def closure():
    optimizer.zero_grad()
    logits = model(x, adj_t)
    # For node classification: logits shape [N, num_classes]
    loss = criterion(logits, labels)
    loss.backward()
    return loss

optimizer = RiemannianLBFGS(model.parameters(), lr=0.5, history_size=10, c=1.0)

# Training loop
for epoch in range(20):
    loss = optimizer.step(closure)
    print(f"Epoch {epoch}, Loss: {loss.item()}")


<ipython-input-6-02a3bb11f621>:61: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  norm_adj = torch.tensor(D_inv_sqrt @ adj @ D_inv_sqrt, dtype=torch.float32)


TypeError: Stereographic.expmap() got an unexpected keyword argument 'c'

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import networkx as nx
from torch import optim
import geoopt
from geoopt import ManifoldParameter

##############################################
# Data Loading (Example)
##############################################

def load_fb15k_data(train_path, valid_path, test_path):
    def read_triples(path, ent2id, rel2id):
        triples = []
        with open(path, 'r') as f:
            for line in f:
                h, r, t = line.strip().split('\t')
                if h not in ent2id:
                    ent2id[h] = len(ent2id)
                if t not in ent2id:
                    ent2id[t] = len(ent2id)
                if r not in rel2id:
                    rel2id[r] = len(rel2id)
                triples.append((h, r, t))
        return triples

    ent2id = {}
    rel2id = {}
    train_triples = read_triples(train_path, ent2id, rel2id)
    valid_triples = read_triples(valid_path, ent2id, rel2id)
    test_triples = read_triples(test_path, ent2id, rel2id)

    G = nx.Graph()
    G.add_nodes_from(range(len(ent2id)))
    for h, r, t in train_triples:
        G.add_edge(ent2id[h], ent2id[t])

    return ent2id, rel2id, G, train_triples, valid_triples, test_triples

# Example paths (You must provide actual paths)
train_path = 'train.txt'
valid_path = 'valid.txt'
test_path = 'test.txt'

ent2id, rel2id, G, train_triples, valid_triples, test_triples = load_fb15k_data(train_path, valid_path, test_path)

num_nodes = len(ent2id)
adj = nx.to_scipy_sparse_array(G, nodelist=range(num_nodes))
adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)  # symmetrize
adj = torch.tensor(adj.toarray(), dtype=torch.float32) + 1e-5*torch.eye(num_nodes)
degrees = adj.sum(axis=1)
inv_sqrt_deg = 1. / torch.sqrt(degrees)
inv_sqrt_deg[torch.isinf(inv_sqrt_deg)] = 0
D_inv_sqrt = torch.diag(inv_sqrt_deg)
norm_adj = torch.tensor(D_inv_sqrt @ adj @ D_inv_sqrt, dtype=torch.float32)

# Synthetic labels for demonstration (e.g., node classification)
num_classes = 5
labels = torch.randint(0, num_classes, (num_nodes,))

# Features: using identity for simplicity
features = torch.eye(num_nodes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
features, labels, norm_adj = features.to(device), labels.to(device), norm_adj.to(device)

##############################################
# Hyperbolic GCN Layers
##############################################

class HyperbolicLinear(nn.Module):
    """
    Hyperbolic linear layer using the Poincaré Ball model.
    """
    def __init__(self, manifold, in_features, out_features, c=1.0, bias=True):
        super(HyperbolicLinear, self).__init__()
        self.manifold = manifold
        self.in_features = in_features
        self.out_features = out_features
        self.c = c
        # ManifoldParameter ensures the parameter stays on the manifold
        # Here weight is in tangent space initially; we keep it Euclidean and apply expmap0 as needed.
        self.weight = ManifoldParameter(torch.randn(out_features, in_features)*0.01, manifold=geoopt.Euclidean())
        if bias:
            self.bias = nn.Parameter(torch.zeros(out_features))
        else:
            self.register_parameter('bias', None)

    def forward(self, x):
        # x is on manifold. We first map to tangent space at 0, apply linear, then map back.
        x_tan = self.manifold.logmap0(x, dim=-1)
        out = F.linear(x_tan, self.weight, self.bias)
        out = self.manifold.expmap0(out, dim=-1)
        return out


class HyperbolicGCNLayer(nn.Module):
    def __init__(self, manifold, in_features, out_features, c=1.0):
        super(HyperbolicGCNLayer, self).__init__()
        self.manifold = manifold
        self.lin = HyperbolicLinear(manifold, in_features, out_features, c=c)
        self.c = c

    def forward(self, x, adj):
        # x on manifold.
        # Aggregate neighbors in tangent space:
        x_tan = self.manifold.logmap0(x, dim=-1)
        x_agg_tan = adj @ x_tan  # linear combination in tangent space
        x_agg = self.manifold.expmap0(x_agg_tan, dim=-1)
        # Apply hyperbolic linear
        return self.lin(x_agg)


class HyperbolicGCN(nn.Module):
    def __init__(self, manifold, num_features, hidden_dim, num_classes, c=1.0):
        super(HyperbolicGCN, self).__init__()
        self.manifold = manifold
        self.c = c
        self.layer1 = HyperbolicGCNLayer(manifold, num_features, hidden_dim, c=c)
        self.layer2 = HyperbolicGCNLayer(manifold, hidden_dim, num_classes, c=c)

    def forward(self, x, adj):
        x = self.manifold.expmap0(x, dim=-1)  # ensure x is on manifold
        x = self.layer1(x, adj)
        # Hyperbolic activation: apply tanh in tangent space
        x_tan = self.manifold.logmap0(x, dim=-1)
        x_tan = torch.tanh(x_tan)
        x = self.manifold.expmap0(x_tan, dim=-1)
        x = self.layer2(x, adj)
        return x


##############################################
# Custom Riemannian L-BFGS
##############################################

class RiemannianLBFGS(optim.Optimizer):
    """
    A conceptual Riemannian L-BFGS optimizer using geoopt.
    """
    def __init__(self, params, lr=1.0, history_size=10, c=1.0):
        defaults = dict(lr=lr, history_size=history_size, c=c)
        super(RiemannianLBFGS, self).__init__(params, defaults)
        for group in self.param_groups:
            group['s_history'] = []
            group['y_history'] = []
            group['rho_history'] = []

    def step(self, closure=None):
        loss = None
        if closure is not None:
            with torch.enable_grad():
                loss = closure()

        for group in self.param_groups:
            c = group['c']
            lr = group['lr']
            s_history = group['s_history']
            y_history = group['y_history']
            rho_history = group['rho_history']
            history_size = group['history_size']

            # Gather parameters and gradients
            params_data = []
            grads = []
            manifolds = []
            for p in group['params']:
                if p.grad is None:
                    continue
                x = p.data
                # Convert Euclidean grad to Riemannian grad
                # egrad2rgrad requires point on manifold and Euclidean grad
                # If p is a ManifoldParameter, p.manifold is the associated manifold
                # If no manifold is set for param, assume Euclidean
                m = getattr(p, 'manifold', geoopt.Euclidean())
                manifolds.append((p, m))
                p.grad = m.egrad2rgrad(x, p.grad)  # Riemannian gradient
                params_data.append(x.view(-1))
                grads.append(p.grad.view(-1))
            if len(grads) == 0:
                continue
            flat_params = torch.cat(params_data)
            grad = torch.cat(grads)

            # Initial direction is -grad
            direction = -grad.clone()

            # Two-loop recursion if we have history
            if len(s_history) > 0:
                alpha = []
                q = grad.clone()
                for (s, y, rho) in reversed(list(zip(s_history, y_history, rho_history))):
                    alpha_i = rho * torch.dot(s, q)
                    q = q - alpha_i * y
                    alpha.append(alpha_i)
                # Scale by H0
                if len(y_history) > 0:
                    y_last = y_history[-1]
                    s_last = s_history[-1]
                    gamma = torch.dot(s_last, y_last) / torch.dot(y_last, y_last)
                else:
                    gamma = 1.0
                r = gamma * q
                for (s, y, rho, alpha_i) in zip(s_history, y_history, rho_history, reversed(alpha)):
                    beta = rho * torch.dot(y, r)
                    r = r + s * (alpha_i - beta)
                direction = -r

            # Update parameters on the manifold
            start_params = flat_params
            offset = 0
            for (p, m) in manifolds:
                sz = p.numel()
                dp = direction[offset:offset+sz].view_as(p.data)
                offset += sz
                # Move along geodesic using expmap
                # p_new = expmap_x(u), where x = p.data and u = dp is tangent vector
                # dp must be projected onto tangent space first
                dp = m.proju(p.data, dp)
                p.data = m.expmap(p.data, dp)  # Move p along dp

            # Recompute gradients for y calculation
            new_grads = []
            if closure is not None:
                new_loss = closure()
            else:
                # If no closure, we assume no update of s,y.
                # In practice, you must provide a closure for LBFGS
                new_loss = None

            offset = 0
            for (p, m) in manifolds:
                if p.grad is None:
                    # If no grad, can't form y
                    continue
                new_grads.append(p.grad.view(-1))
            if len(new_grads) == 0:
                continue
            new_grad = torch.cat(new_grads)

            end_params = torch.cat([p.data.view(-1) for (p,m) in manifolds])
            s_k = (end_params - start_params).detach()
            y_k = (new_grad - grad).detach()

            # Update histories
            s_history.insert(0, s_k)
            y_history.insert(0, y_k)
            rho = 1.0 / torch.dot(y_k, s_k)
            rho_history.insert(0, rho)
            if len(s_history) > history_size:
                s_history.pop()
                y_history.pop()
                rho_history.pop()

        return loss


##############################################
# Model and Training
##############################################

manifold = geoopt.manifolds.PoincareBall(c=1.0)

model = HyperbolicGCN(manifold, num_features=num_nodes, hidden_dim=64, num_classes=num_classes, c=1.0).to(device)

# Initialize input in tangent space and map to manifold
x = manifold.expmap0(features, dim=-1)

criterion = nn.CrossEntropyLoss()

def closure():
    optimizer.zero_grad()
    logits = model(x, norm_adj)
    loss = criterion(logits, labels)
    loss.backward()
    return loss

optimizer = RiemannianLBFGS(model.parameters(), lr=0.5, history_size=10, c=1.0)

for epoch in range(10):
    loss = optimizer.step(closure)
    print(f"Epoch {epoch}, Loss: {loss.item()}")


<ipython-input-7-b1425d37635a>:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  norm_adj = torch.tensor(D_inv_sqrt @ adj @ D_inv_sqrt, dtype=torch.float32)


Epoch 0, Loss: 1.6094367504119873
Epoch 1, Loss: 1.6094053983688354
Epoch 2, Loss: 1.6093485355377197
Epoch 3, Loss: 1.6093485355377197
Epoch 4, Loss: 1.6093387603759766
Epoch 5, Loss: 1.609322190284729
Epoch 6, Loss: 1.6093178987503052
Epoch 7, Loss: 1.6093202829360962
Epoch 8, Loss: 1.609328031539917
Epoch 9, Loss: 1.6093600988388062


In [8]:
torch.version

<module 'torch.version' from '/usr/local/lib/python3.10/dist-packages/torch/version.py'>

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import networkx as nx
from torch import optim
import geoopt
from geoopt import ManifoldParameter

# Assuming you've loaded FB15K data into train_triples, etc.
# and built norm_adj and features as before.

class HyperbolicLinear(nn.Module):
    def __init__(self, manifold, in_features, out_features, bias=True):
        super(HyperbolicLinear, self).__init__()
        self.manifold = manifold
        self.in_features = in_features
        self.out_features = out_features
        self.weight = ManifoldParameter(torch.randn(out_features, in_features)*0.01, manifold=geoopt.Euclidean())
        if bias:
            self.bias = nn.Parameter(torch.zeros(out_features))
        else:
            self.register_parameter('bias', None)

    def forward(self, x):
        x_tan = self.manifold.logmap0(x, dim=-1)
        out = F.linear(x_tan, self.weight, self.bias)
        out = self.manifold.expmap0(out, dim=-1)
        return out

class HyperbolicGCNLayer(nn.Module):
    def __init__(self, manifold, in_features, out_features):
        super(HyperbolicGCNLayer, self).__init__()
        self.manifold = manifold
        self.lin = HyperbolicLinear(manifold, in_features, out_features)

    def forward(self, x, adj):
        x_tan = self.manifold.logmap0(x, dim=-1)
        x_agg_tan = adj @ x_tan
        x_agg = self.manifold.expmap0(x_agg_tan, dim=-1)
        return self.lin(x_agg)

class HyperbolicGCN(nn.Module):
    def __init__(self, manifold, num_features, hidden_dim, num_classes):
        super(HyperbolicGCN, self).__init__()
        self.manifold = manifold
        self.layer1 = HyperbolicGCNLayer(manifold, num_features, hidden_dim)
        self.layer2 = HyperbolicGCNLayer(manifold, hidden_dim, num_classes)

    def forward(self, x, adj):
        x = self.manifold.expmap0(x, dim=-1)
        x = self.layer1(x, adj)
        x_tan = self.manifold.logmap0(x, dim=-1)
        x_tan = torch.tanh(x_tan)
        x = self.manifold.expmap0(x_tan, dim=-1)
        x = self.layer2(x, adj)
        return x

# Make the curvature learnable
manifold = geoopt.PoincareBall(c=1.0, learnable=True)

model = HyperbolicGCN(manifold, num_features=num_nodes, hidden_dim=64, num_classes=num_classes).to(device)

# Initialize input
x = manifold.expmap0(features, dim=-1)

criterion = nn.CrossEntropyLoss()

def closure():
    optimizer.zero_grad()
    logits = model(x, norm_adj)
    loss = criterion(logits, labels)
    loss.backward(retain_graph=True)
    return loss

# Use a smaller LR, consider using RiemannianAdam for more stability
optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(10):
    loss = closure()
    optimizer.step()
    print(f"Epoch {epoch}, Loss: {loss.item()}, Curvature: {manifold.c.item()}")

# After training, you can inspect manifold.c to see if curvature adjusted.


Epoch 0, Loss: 1.609438180923462, Curvature: 1.0005226135253906


RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.cuda.FloatTensor []] is at version 4; expected version 3 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import networkx as nx
from torch import optim
import geoopt
from geoopt import ManifoldParameter

##############################################
# Data Loading (Example)
##############################################

def load_fb15k_data(train_path, valid_path, test_path):
    def read_triples(path, ent2id, rel2id):
        triples = []
        with open(path, 'r') as f:
            for line in f:
                h, r, t = line.strip().split('\t')
                if h not in ent2id:
                    ent2id[h] = len(ent2id)
                if t not in ent2id:
                    ent2id[t] = len(ent2id)
                if r not in rel2id:
                    rel2id[r] = len(rel2id)
                triples.append((ent2id[h], rel2id[r], ent2id[t]))
        return triples

    ent2id = {}
    rel2id = {}
    train_triples = read_triples(train_path, ent2id, rel2id)
    valid_triples = read_triples(valid_path, ent2id, rel2id)
    test_triples = read_triples(test_path, ent2id, rel2id)

    G = nx.Graph()
    G.add_nodes_from(range(len(ent2id)))
    for h, r, t in train_triples:
        G.add_edge(h, t)

    return ent2id, rel2id, G, train_triples, valid_triples, test_triples

# Example paths (You must provide actual paths)
train_path = 'train.txt'
valid_path = 'valid.txt'
test_path = 'test.txt'

# For demonstration, create dummy data if files are not available
import os
if not os.path.exists(train_path):
    with open(train_path, 'w') as f:
        for i in range(100):
            f.write(f"entity_{i}\trelation_{i%10}\tentity_{(i+1)%100}\n")
    with open(valid_path, 'w') as f:
        for i in range(100, 120):
            f.write(f"entity_{i%100}\trelation_{i%10}\tentity_{(i+1)%100}\n")
    with open(test_path, 'w') as f:
        for i in range(120, 140):
            f.write(f"entity_{i%100}\trelation_{i%10}\tentity_{(i+1)%100}\n")

ent2id, rel2id, G, train_triples, valid_triples, test_triples = load_fb15k_data(train_path, valid_path, test_path)

num_nodes = len(ent2id)
num_relations = len(rel2id)

adj = nx.to_scipy_sparse_array(G, nodelist=range(num_nodes))
adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)  # symmetrize
adj = torch.tensor(adj.toarray(), dtype=torch.float32) + 1e-5*torch.eye(num_nodes)
degrees = adj.sum(axis=1)
inv_sqrt_deg = 1. / torch.sqrt(degrees)
inv_sqrt_deg[torch.isinf(inv_sqrt_deg)] = 0
D_inv_sqrt = torch.diag(inv_sqrt_deg)
norm_adj = D_inv_sqrt @ adj @ D_inv_sqrt
norm_adj = norm_adj.to(torch.float32)

# Node features: using identity for simplicity
features = torch.eye(num_nodes, device='cpu')  # Initialize on CPU first

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
features, norm_adj = features.to(device), norm_adj.to(device)

##############################################
# Hyperbolic GCN Layers
##############################################

class HyperbolicLinear(nn.Module):
    """
    Hyperbolic linear layer using the Poincaré Ball model.
    """
    def __init__(self, manifold, in_features, out_features, bias=True):
        super(HyperbolicLinear, self).__init__()
        self.manifold = manifold
        self.in_features = in_features
        self.out_features = out_features
        # ManifoldParameter for weight in Euclidean space
        self.weight = ManifoldParameter(torch.randn(out_features, in_features)*0.01, manifold=geoopt.Euclidean())
        if bias:
            self.bias = nn.Parameter(torch.zeros(out_features))
        else:
            self.register_parameter('bias', None)

    def forward(self, x):
        # x is on manifold. Map to tangent space at 0, apply linear, map back
        x_tan = self.manifold.logmap0(x, dim=-1)
        out = F.linear(x_tan, self.weight, self.bias)
        out = self.manifold.expmap0(out, dim=-1)
        return out

class HyperbolicGCNLayer(nn.Module):
    def __init__(self, manifold, in_features, out_features):
        super(HyperbolicGCNLayer, self).__init__()
        self.manifold = manifold
        self.lin = HyperbolicLinear(manifold, in_features, out_features)

    def forward(self, x, adj):
        # x on manifold
        x_tan = self.manifold.logmap0(x, dim=-1)
        x_agg_tan = adj @ x_tan  # aggregate in tangent space
        x_agg = self.manifold.expmap0(x_agg_tan, dim=-1)
        return self.lin(x_agg)

class HyperbolicGCN(nn.Module):
    def __init__(self, manifold, num_features, hidden_dim, num_classes):
        super(HyperbolicGCN, self).__init__()
        self.manifold = manifold
        self.layer1 = HyperbolicGCNLayer(manifold, num_features, hidden_dim)
        self.layer2 = HyperbolicGCNLayer(manifold, hidden_dim, num_classes)

    def forward(self, x, adj):
        x = self.manifold.expmap0(x, dim=-1)  # ensure x is on manifold
        x = self.layer1(x, adj)
        # Hyperbolic activation: apply tanh in tangent space
        x_tan = self.manifold.logmap0(x, dim=-1)
        x_tan = torch.tanh(x_tan)
        x = self.manifold.expmap0(x_tan, dim=-1)
        x = self.layer2(x, adj)
        return x

##############################################
# Model and Training
##############################################

# Make the curvature learnable
manifold = geoopt.PoincareBall(c=1.0, learnable=True)

model = HyperbolicGCN(manifold, num_features=num_nodes, hidden_dim=64, num_classes=num_relations).to(device)

# Initialize input: ensure features are on the manifold
x = manifold.expmap0(features, dim=-1)

# Define optimizer using Geoopt's RiemannianAdam
optimizer = geoopt.optim.RiemannianAdam(
    [
        {'params': model.parameters()},
        {'params': [manifold.c], 'lr': 1e-3}  # Add 'stabilize' with a value
    ],
    lr=1e-3,
    stabilize=0
)

# Define loss function for link prediction: Binary Cross Entropy with logits
criterion = nn.BCEWithLogitsLoss()

##############################################
# Link Prediction Training Loop
##############################################

def get_positive_negative_samples(train_triples, num_nodes, num_neg_samples=5):
    positive = set(train_triples)
    negatives = []
    while len(negatives) < len(train_triples) * num_neg_samples:
        h = torch.randint(0, num_nodes, (1,)).item()
        t = torch.randint(0, num_nodes, (1,)).item()
        r = torch.randint(0, num_relations, (1,)).item()
        if (h, r, t) not in positive:
            negatives.append((h, r, t))
    return train_triples, negatives

train_pos, train_neg = get_positive_negative_samples(train_triples, num_nodes)

# Convert to tensors
train_pos = torch.tensor(train_pos, dtype=torch.long, device=device)
train_neg = torch.tensor(train_neg, dtype=torch.long, device=device)

def train_epoch(model, optimizer, criterion, pos_triples, neg_triples):
    model.train()
    optimizer.zero_grad()

    # Combine positive and negative samples
    pos_labels = torch.ones(pos_triples.size(0), device=device)
    neg_labels = torch.zeros(neg_triples.size(0), device=device)
    triples = torch.cat([pos_triples, neg_triples], dim=0)
    labels = torch.cat([pos_labels, neg_labels], dim=0).unsqueeze(1)  # Shape: [2N, 1]

    # Get embeddings
    heads = triples[:,0]
    relations = triples[:,1]
    tails = triples[:,2]

    head_emb = model.manifold.logmap0(model.manifold.expmap0(x, dim=-1)[heads], dim=-1)
    tail_emb = model.manifold.logmap0(model.manifold.expmap0(x, dim=-1)[tails], dim=-1)

    # For simplicity, ignore relations or handle them appropriately
    # Here, we'll just use head and tail embeddings with a simple dot product
    # A proper implementation should incorporate relation embeddings

    # Compute score: inner product in tangent space
    scores = (head_emb * tail_emb).sum(dim=-1, keepdim=True)  # Shape: [2N, 1]

    loss = criterion(scores, labels)
    loss.backward()
    optimizer.step()

    return loss.item()

# Enable anomaly detection to help debug further errors
torch.autograd.set_detect_anomaly(True)

num_epochs = 10
for epoch in range(num_epochs):
    loss = train_epoch(model, optimizer, criterion, train_pos, train_neg)
    print(f"Epoch {epoch}, Loss: {loss:.4f}, Curvature: {manifold.c.item():.6f}")


ValueError: can't optimize a non-leaf Tensor

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import networkx as nx
from torch import optim
import geoopt
from geoopt import ManifoldParameter

##############################################
# Data Loading (Example)
##############################################

def load_fb15k_data(train_path, valid_path, test_path):
    def read_triples(path, ent2id, rel2id):
        triples = []
        with open(path, 'r') as f:
            for line in f:
                h, r, t = line.strip().split('\t')
                if h not in ent2id:
                    ent2id[h] = len(ent2id)
                if t not in ent2id:
                    ent2id[t] = len(ent2id)
                if r not in rel2id:
                    rel2id[r] = len(rel2id)
                triples.append((ent2id[h], rel2id[r], ent2id[t]))
        return triples

    ent2id = {}
    rel2id = {}
    train_triples = read_triples(train_path, ent2id, rel2id)
    valid_triples = read_triples(valid_path, ent2id, rel2id)
    test_triples = read_triples(test_path, ent2id, rel2id)

    G = nx.Graph()
    G.add_nodes_from(range(len(ent2id)))
    for h, r, t in train_triples:
        G.add_edge(h, t)

    return ent2id, rel2id, G, train_triples, valid_triples, test_triples

# Example paths (You must provide actual paths)
train_path = 'train.txt'
valid_path = 'valid.txt'
test_path = 'test.txt'

# For demonstration, create dummy data if files are not available
import os
if not os.path.exists(train_path):
    with open(train_path, 'w') as f:
        for i in range(100):
            f.write(f"entity_{i}\trelation_{i%10}\tentity_{(i+1)%100}\n")
    with open(valid_path, 'w') as f:
        for i in range(100, 120):
            f.write(f"entity_{i%100}\trelation_{i%10}\tentity_{(i+1)%100}\n")
    with open(test_path, 'w') as f:
        for i in range(120, 140):
            f.write(f"entity_{i%100}\trelation_{i%10}\tentity_{(i+1)%100}\n")

ent2id, rel2id, G, train_triples, valid_triples, test_triples = load_fb15k_data(train_path, valid_path, test_path)

num_nodes = len(ent2id)
num_relations = len(rel2id)

adj = nx.to_scipy_sparse_array(G, nodelist=range(num_nodes))
adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)  # symmetrize
adj = torch.tensor(adj.toarray(), dtype=torch.float32) + 1e-5*torch.eye(num_nodes)
degrees = adj.sum(axis=1)
inv_sqrt_deg = 1. / torch.sqrt(degrees)
inv_sqrt_deg[torch.isinf(inv_sqrt_deg)] = 0
D_inv_sqrt = torch.diag(inv_sqrt_deg)
norm_adj = D_inv_sqrt @ adj @ D_inv_sqrt
norm_adj = norm_adj.to(torch.float32)

# Node features: using identity for simplicity
features = torch.eye(num_nodes, device='cpu')  # Initialize on CPU first

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
features, norm_adj = features.to(device), norm_adj.to(device)

##############################################
# Hyperbolic GCN Layers
##############################################

class HyperbolicLinear(nn.Module):
    """
    Hyperbolic linear layer using the Poincaré Ball model.
    """
    def __init__(self, manifold, in_features, out_features, bias=True):
        super(HyperbolicLinear, self).__init__()
        self.manifold = manifold
        self.in_features = in_features
        self.out_features = out_features
        # ManifoldParameter for weight in Euclidean space
        self.weight = ManifoldParameter(torch.randn(out_features, in_features)*0.01, manifold=geoopt.Euclidean())
        if bias:
            self.bias = nn.Parameter(torch.zeros(out_features))
        else:
            self.register_parameter('bias', None)

    def forward(self, x):
        # x is on manifold. Map to tangent space at 0, apply linear, map back
        x_tan = self.manifold.logmap0(x, dim=-1)
        out = F.linear(x_tan, self.weight, self.bias)
        out = self.manifold.expmap0(out, dim=-1)
        return out

class HyperbolicGCNLayer(nn.Module):
    def __init__(self, manifold, in_features, out_features):
        super(HyperbolicGCNLayer, self).__init__()
        self.manifold = manifold
        self.lin = HyperbolicLinear(manifold, in_features, out_features)

    def forward(self, x, adj):
        # x on manifold
        x_tan = self.manifold.logmap0(x, dim=-1)
        x_agg_tan = adj @ x_tan  # aggregate in tangent space
        x_agg = self.manifold.expmap0(x_agg_tan, dim=-1)
        return self.lin(x_agg)

class HyperbolicGCN(nn.Module):
    def __init__(self, num_features, hidden_dim, num_classes, c=1.0):
        super(HyperbolicGCN, self).__init__()
        # Initialize the manifold inside the model to ensure its parameters are included
        self.manifold = geoopt.PoincareBall(c=c, learnable=True)
        self.layer1 = HyperbolicGCNLayer(self.manifold, num_features, hidden_dim)
        self.layer2 = HyperbolicGCNLayer(self.manifold, hidden_dim, num_classes)

    def forward(self, x, adj):
        x = self.manifold.expmap0(x, dim=-1)  # ensure x is on manifold
        x = self.layer1(x, adj)
        # Hyperbolic activation: apply tanh in tangent space
        x_tan = self.manifold.logmap0(x, dim=-1)
        x_tan = torch.tanh(x_tan)
        x = self.manifold.expmap0(x_tan, dim=-1)
        x = self.layer2(x, adj)
        return x

##############################################
# Link Prediction Utilities
##############################################

def get_positive_negative_samples(train_triples, num_nodes, num_neg_samples=5):
    positive = set(train_triples)
    negatives = []
    while len(negatives) < len(train_triples) * num_neg_samples:
        h = torch.randint(0, num_nodes, (1,)).item()
        t = torch.randint(0, num_nodes, (1,)).item()
        r = torch.randint(0, num_relations, (1,)).item()
        if (h, r, t) not in positive:
            negatives.append((h, r, t))
    return train_triples, negatives

def train_epoch(model, optimizer, criterion, pos_triples, neg_triples):
    model.train()
    optimizer.zero_grad()

    # Combine positive and negative samples
    pos_labels = torch.ones(pos_triples.size(0), device=device)
    neg_labels = torch.zeros(neg_triples.size(0), device=device)
    triples = torch.cat([pos_triples, neg_triples], dim=0)
    labels = torch.cat([pos_labels, neg_labels], dim=0).unsqueeze(1)  # Shape: [2N, 1]

    # Get embeddings
    heads = triples[:,0]
    relations = triples[:,1]
    tails = triples[:,2]

    # Forward pass through the model
    logits = model(features, norm_adj)  # Shape: [num_nodes, num_relations]

    # For link prediction, define a scoring function
    # Here, we'll use a simple dot product between head and tail embeddings
    # A proper implementation should incorporate relation embeddings
    # For demonstration, we'll compute scores for each triple
    head_emb = logits[heads]  # Shape: [2N, num_classes]
    tail_emb = logits[tails]  # Shape: [2N, num_classes]

    # Simple dot product scoring
    scores = (head_emb * tail_emb).sum(dim=1, keepdim=True)  # Shape: [2N, 1]

    loss = criterion(scores, labels)
    loss.backward()
    optimizer.step()

    return loss.item()

##############################################
# Model and Training
##############################################

model = HyperbolicGCN(num_features=num_nodes, hidden_dim=64, num_classes=num_relations, c=0.0001).to(device)

# Initialize input: ensure features are on the manifold
# Here, features are identity; map them to the manifold
x = model.manifold.expmap0(features, dim=-1)

# Define optimizer using Geoopt's RiemannianAdam
optimizer = geoopt.optim.RiemannianAdam(model.parameters(), lr=1e-3)

# Define loss function for link prediction: Binary Cross Entropy with logits
criterion = nn.BCEWithLogitsLoss()

# Prepare training samples
train_pos, train_neg = get_positive_negative_samples(train_triples, num_nodes)

# Convert to tensors
train_pos = torch.tensor(train_pos, dtype=torch.long, device=device)
train_neg = torch.tensor(train_neg, dtype=torch.long, device=device)

# Enable anomaly detection to help debug further errors
torch.autograd.set_detect_anomaly(True)

num_epochs = 100
for epoch in range(num_epochs):
    loss = train_epoch(model, optimizer, criterion, train_pos, train_neg)
    print(f"Epoch {epoch}, Loss: {loss:.4f}, Curvature: {model.manifold.c.item():.6f}")


Epoch 0, Loss: 0.6931, Curvature: 0.000100
Epoch 1, Loss: 0.6932, Curvature: 0.000100
Epoch 2, Loss: 0.6932, Curvature: 0.000100
Epoch 3, Loss: 0.6932, Curvature: 0.000100
Epoch 4, Loss: 0.6932, Curvature: 0.000100
Epoch 5, Loss: 0.6932, Curvature: 0.000100
Epoch 6, Loss: 0.6932, Curvature: 0.000100
Epoch 7, Loss: 0.6931, Curvature: 0.000100
Epoch 8, Loss: 0.6932, Curvature: 0.000100
Epoch 9, Loss: 0.6932, Curvature: 0.000100
Epoch 10, Loss: 0.6932, Curvature: 0.000100
Epoch 11, Loss: 0.6931, Curvature: 0.000100


In [1]:
!pip install geoopt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 8.2 MB/s eta 0:00:00
